In [48]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.dates as mdates

In [49]:
# 1. 抓取股價資料
def get_stock_data(stock_id, start_date, end_date):
    df = yf.download(stock_id, start=start_date, end=end_date)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
    return df

stock_data = get_stock_data('2330.TW', '2020-01-01', '2023-01-01')
stock_data.head(10)

[*********************100%***********************]  1 of 1 completed


,開盤價,最高價,最低價,收盤價,成交量
Date,,,,,
2020-01-02,332.5,339.0,332.5,339.0,31754120
2020-01-03,344.0,345.0,335.5,339.5,41811268
2020-01-06,333.0,334.5,332.0,332.0,45343057
2020-01-07,332.5,333.0,326.5,329.5,50879181
2020-01-08,325.0,333.0,325.0,329.5,37567748
2020-01-09,335.0,337.5,333.5,337.5,31481504
2020-01-10,340.5,341.0,336.0,339.5,27032115
2020-01-13,342.0,342.0,339.0,341.5,30663332
2020-01-14,345.5,346.0,344.5,346.0,30223993


In [50]:
# 2. 標記漲跌標籤
def create_labels(df):
    df['5_SMA'] = df['收盤價'].rolling(window=5).mean()
    df['10_SMA'] = df['收盤價'].rolling(window=10).mean()
    df['Label'] = np.where(df['5_SMA'] > df['10_SMA'], 1, 0)  # 漲為1，跌為0
    df.dropna(inplace=True)  # 移除 NaN
    return df

df = create_labels(stock_data)
df.head(10)

,開盤價,最高價,最低價,收盤價,成交量,5_SMA,10_SMA,Label
Date,,,,,,,,
2020-01-15,345.0,345.0,337.5,340.0,47434274,340.9,337.40,1
2020-01-16,330.0,336.5,330.0,334.5,55211420,340.3,336.95,1
2020-01-17,334.0,335.5,332.0,333.0,52060097,339.0,336.30,1
2020-01-20,334.0,335.5,333.0,333.0,32658203,337.3,336.40,1
2020-01-30,326.0,326.0,316.5,316.5,125451049,331.4,335.10,0
2020-01-31,323.0,323.5,319.0,320.0,62526055,327.4,334.15,0
2020-02-03,315.0,316.5,312.0,315.0,59560849,323.5,331.90,0
2020-02-04,336.5,337.0,325.0,325.0,51890496,321.9,330.45,0
2020-02-05,329.0,329.5,324.5,327.5,36601453,320.8,329.05,0


In [31]:
# 3. 保存 K 線圖
def save_candlestick_images(df, output_dir='candlestick_images'):
    os.makedirs(output_dir, exist_ok=True)

    for i in range(20, len(df)):
        subset = df.iloc[i-20:i]
        label = df.iloc[i]['Label']
        label_str = 'up' if label == 1 else 'down'
        save_path = os.path.join(output_dir, f"candlestick_{i}_{label_str}.png")  # 文件名
        try:
            fig, ax = plt.subplots(figsize=(6, 4))
            for idx, row in subset.iterrows():
                date = mdates.date2num(idx)
                open_price, high, low, close = row['開盤價'], row['最高價'], row['最低價'], row['收盤價']
                color = 'green' if close >= open_price else 'red'

                rect = plt.Rectangle((date - 0.2, min(open_price, close)), 0.4, abs(close - open_price), color=color)
                ax.add_patch(rect)
                plt.plot([date, date], [low, high], color=color)

            ax.xaxis_date()
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            plt.xticks(rotation=45)
            plt.axis('off')
            plt.grid(False)

            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close(fig)
        except Exception as e:
            print(f"Failed to save candlestick image for index {i}: {e}")

# 呼叫函數保存 K 線圖
save_candlestick_images(df)

In [55]:
# 4. 定義 Dataset 和 DataLoader
# 定義 Dataset
class KLineDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index()  # 確保索引從 0 開始
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df) - 20  # 減去 20，因為從第 20 行開始生成圖像

    def __getitem__(self, idx):
        idx += 20  # 調整索引匹配圖像文件名
        row = self.df.iloc[idx]
        label = row['Label']
        img_path = os.path.join(self.img_dir, f"candlestick_{idx}_{'up' if label == 1 else 'down'}.png")

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image file not found: {img_path}")

        # 加載圖像
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        # 數值特徵
        numeric_features = torch.tensor(
            [row['開盤價'], row['最高價'], row['最低價'], row['成交量']], dtype=torch.float32
        )

        return img, numeric_features, torch.tensor(label, dtype=torch.long)

# 設置圖像變換
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# 創建數據集
dataset = KLineDataset(df, img_dir='candlestick_images', transform=transform)

# 分割訓練集和測試集
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 創建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [56]:
class MultiModalModel(nn.Module):
    def __init__(self):
        super(MultiModalModel, self).__init__()
        # CNN 模型
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # 去掉 ResNet 的全連接層

        # 數值特徵的全連接層
        self.fc_numeric = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
        )

        # 融合層
        self.fc_combined = nn.Sequential(
            nn.Linear(512 + 32, 64),
            nn.ReLU(),
            nn.Linear(64, 2)  # 2分類
        )

    def forward(self, img, numeric_features):
        # CNN 處理圖像
        img_features = self.cnn(img)
        
        # 全連接層處理數值特徵
        numeric_features = self.fc_numeric(numeric_features)

        # 融合圖像特徵和數值特徵
        combined = torch.cat((img_features, numeric_features), dim=1)

        # 融合特徵進行分類
        output = self.fc_combined(combined)
        return output

In [57]:
# 6. 模型訓練與測試
# 初始化模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiModalModel().to(device)

# 設置損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

MultiModalModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

c:\Users\User\miniconda3\envs\pytorch_transformer_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\miniconda3\envs\pytorch_transformer_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [58]:
def train_model(model, train_loader, test_loader, epochs=50):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for imgs, numeric_features, labels in train_loader:
            imgs, numeric_features, labels = imgs.to(device), numeric_features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs, numeric_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}")

        # 測試模型
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for imgs, numeric_features, labels in test_loader:
                imgs, numeric_features, labels = imgs.to(device), numeric_features.to(device), labels.to(device)
                outputs = model(imgs, numeric_features)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        print(f"Test Accuracy: {accuracy:.4f}")

# 開始訓練模型
train_model(model, train_loader, test_loader, epochs=10)

Epoch 1/10, Loss: 8.9965
Test Accuracy: 0.8652
Epoch 2/10, Loss: 4.7225
Test Accuracy: 0.8794
Epoch 3/10, Loss: 2.5174
Test Accuracy: 0.8865
Epoch 4/10, Loss: 3.0050
Test Accuracy: 0.8440
Epoch 5/10, Loss: 3.0581
Test Accuracy: 0.8652
Epoch 6/10, Loss: 2.4794
Test Accuracy: 0.8865
Epoch 7/10, Loss: 2.5723
Test Accuracy: 0.9149
Epoch 8/10, Loss: 1.0778
Test Accuracy: 0.8936
Epoch 9/10, Loss: 0.5739
Test Accuracy: 0.9078
Epoch 10/10, Loss: 1.5666
Test Accuracy: 0.8652
